In [1]:
import pandas as pd
import numpy as np
import cv2
import pytesseract
import PIL

In [2]:
img = cv2.imread('images/invoice5.png')
cv2.imshow('',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
text = pytesseract.image_to_string(img)

In [4]:
print(text)

INVOICE

East Repair Inc.
1912 Harvest Lane
New York, NY 12210

 

 

BILLTO SHIPTO INVOICE # us-001
John Smith John Smith INVOICE DATE 1110272019
2 Court Square 3787 Pineview Drive Pos

New York, NY 12210 Cambridge, MA 12210 os 2312/2019
DUE DATE 26/02/2019

ay DESCRIPTION UNIT PRICE ‘AMOUNT

1 Front and rear brake cables 100.00 100.00

2 Newset of pedal arms 15.00 30.00

3 Labor Shrs 5.00 1.00

Subtotal 145.00

Sales Tax 6.25% 9.06

TOTAL $154.06

Smith

TERMS & CONDITIONS

Payment is due within 15 days
J hank you Pate ate cack payable: Est Reps



In [5]:
data = pytesseract.image_to_data(img)

In [6]:
#data.split('\n')
datalist = list(map(lambda x:x.split('\t'),data.split('\n')))
df = pd.DataFrame(datalist[1:],columns=datalist[0])

In [7]:
df.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,750,1061,-1,
1,2,1,1,0,0,0,56,48,173,54,-1,
2,3,1,1,1,0,0,56,48,173,54,-1,
3,4,1,1,1,1,0,56,48,173,54,-1,
4,5,1,1,1,1,1,56,48,173,54,96,INVOICE
5,2,1,2,0,0,0,55,130,117,52,-1,
6,3,1,2,1,0,0,55,130,117,52,-1,
7,4,1,2,1,1,0,55,130,93,13,-1,
8,5,1,2,1,1,1,55,130,26,10,96,East
9,5,1,2,1,1,2,85,130,39,13,96,Repair


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   level      144 non-null    object
 1   page_num   143 non-null    object
 2   block_num  143 non-null    object
 3   par_num    143 non-null    object
 4   line_num   143 non-null    object
 5   word_num   143 non-null    object
 6   left       143 non-null    object
 7   top        143 non-null    object
 8   width      143 non-null    object
 9   height     143 non-null    object
 10  conf       143 non-null    object
 11  text       143 non-null    object
dtypes: object(12)
memory usage: 13.6+ KB


In [9]:
df.dropna(inplace=True)
df_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height','conf']
df[df_int] = df[df_int].astype(int) 

In [10]:
df.dtypes

level         int64
page_num      int64
block_num     int64
par_num       int64
line_num      int64
word_num      int64
left          int64
top           int64
width         int64
height        int64
conf          int64
text         object
dtype: object

In [15]:
image = img.copy()
level = 'word'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    if level=='page':
        if l==1:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
    elif level=='block':
        if l==2:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        else:
            continue
    elif level=='para':
        if l==3:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        else:
            continue
    elif level=='line':
        if l==4:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        else:
            continue
    elif level=='word':
        if l==5:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
cv2.imshow("",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
            